In [2]:
# Create DataSet

from os import listdir
from os.path import isfile, join
import numpy as np
import json

onlyfiles = [f for f in listdir("../dataset/") if isfile(join("../dataset/", f))]

def parse(path: str, pattern_name: str) -> tuple[np.ndarray, np.ndarray]:
    """Парсит уже существующие JSON файл и сортирует координаты в случайном порядке для обучения
    нейронной сети.  

    Args:
        path (str): путь до JSON файла
        pattern_name (str): название паттерна, который будет верным в обучении
    Returns:
        tuple(np.ndarray, np.ndarray): Кортеж из значений на вход (X) и соответствующие им значения корректности
    """
    _X = np.empty((0, 14), int) # Данные для обучения (координаты реконструкции фазового портрета) до сортировки
    _y = np.array([]) # 1 - соответствует правильному паттерну, на который тренируется сеть, 0 - всем остальным до сортировки
    random_value = np.array([]) # Случайная величина для перемешивания датасета
    with open(path, 'r') as json_file:
        data: dict = json.load(json_file)
        
        for ptrn in list(data.keys()):
            all_coordinates: list = data[ptrn]
            
            for local_cordinates in all_coordinates:
                _X = np.append(_X, np.array([np.append(np.array(local_cordinates['x']), np.array(local_cordinates['y'])).tolist()]), axis=0)
                _y = np.append(_y, 1) if ptrn == pattern_name else np.append(_y, 0)
                random_value = np.append(random_value, np.random.rand()) # Генерация случайного числа от 0 до 1

    # Сортировка по случайным величинам
    return (np.array([x for _, x, _ in sorted(zip(random_value, _X, _y), key=lambda x: x[0])]),
            np.array([y for _, _, y in sorted(zip(random_value, _X, _y), key=lambda x: x[0])]))

dataset: list[tuple[tuple, tuple, str]] = list()

training_file_path = '../dataset/26136163268846ea912ade4c2d2b4e5f.json'
test_file_path = '../dataset/e9f2619a28054ec6a081e0bb30822271.json'

patterns = ['Sigmoid', 'SigmoidReversed', 'Normal', 'NormalFlipped', 'Plain', 'LinearIncrease', 'LinearDecrease']

for ptrn in patterns:

    pattern_name = ptrn
    X, y = parse(training_file_path, pattern_name)
    test_data_X, test_data_y = parse(test_file_path, pattern_name)

    dataset += [((X,y),(test_data_X,test_data_y), pattern_name)]

In [6]:
import numpy as np

class Perceptron:
    def __init__(self, lr=0.01, epochs=5):
        self.lr = lr
        self.epochs = epochs
        self.weights = None

    def fit(self, X, y):
        """
        Our fit function trains on the dataset X and tries to predict vector y,
        Using the learning rate, it will modify it's weight vector to increase
        it's accuracy in predictions.
        It will iterate over the X dataset as defined by the epochs.
        Args:
            X: The input data (numpy array of shape [n_samples * m_features])
            y: Class labels vector (numpy array of shape [n_samples])
        """
        # a vector of floats between 0 and 1
        weights = np.random.rand(X.shape[1],)

        for epoch in range(self.epochs):
            # list of predicted classes for our accuracy calculation
            predicted = []
            for i_index, sample in enumerate(X):
                y_hat = self.predict(sample, weights)
                predicted.append(y_hat)  # add our new prediction to the array
                for j_index, feature in enumerate(weights):
                    # update our weight values
                    delta = self.lr * (y[i_index] - y_hat)
                    delta = delta * sample[j_index-1]
                    weights[j_index-1] = weights[j_index-1] + delta
            # print('[Epoch {ep}] Accuracy: {acc}'.format(
            #     ep=epoch, acc=self._calculate_accuracy(y, predicted)
            # ))
        self.weights = weights

    def _calculate_accuracy(self, actual, predicted):
        """
        Calculate the accuracy of predictions for this epoch.
        Args:
            actual: vector of actual class values (the y vector) [n_samples]
            predicted: vector of predicted class values [n_samples]
        """
        return sum(np.array(predicted) == np.array(actual)) / float(len(actual))

    def predict(self, x, w):
        """
        Create a binary prediction from an activation function on the data
        sample and the weight vector.
        Args:
            x: vector of the data sample - shape [m_features]
            w: vector of the weights - shape [m_features]
        Returns:
            0 or 1
        """
        res = self._sum(x, w)
        # print(res)
        return 1 if res > 0.0 else 0.0

    def _sum(self, x, w):
        """
        Multiply our sample and weight vector elements then the sum of the
        result.
        Args:
            x: vector of the data sample - shape [m_features]
            w: vector of the weights - shape [m_features]
        Returns:
            Int of the sum of vector products
        """
        return np.sum(np.dot(x, np.transpose(w)))


if __name__ == '__main__':
    for i in range(5):
        print('-------------------------')
        for data in dataset:
            (_X,_y), (_test_data_X, _test_data_y), name = data
            p = Perceptron()

            # Train
            p.fit(_X, _y)
            truth = 0
            misses = 0
            lies = 0
            for i in range(len(_test_data_X)):
                prd = p.predict(_test_data_X[i],p.weights)
                if prd == y[i]:
                    truth += 1
                elif prd == 0 and y[i] == 1:
                    misses += 1
                elif prd == 1 and y[i] == 0:
                    lies += 1
            print(str(round(truth/len(y),3)) + ' - '+ name + ' Правильное обнаружение')
            print(str(round(misses/len(y),3)) + ' - '+ name + ' Пропуск цели')
            print(str(round(lies/len(y),3)) + ' - '+ name + ' Ложная тревога')
            print('------')
        

-------------------------
0.764 - Sigmoid Правильное обнаружение
0.124 - Sigmoid Пропуск цели
0.112 - Sigmoid Ложная тревога
------
0.856 - SigmoidReversed Правильное обнаружение
0.142 - SigmoidReversed Пропуск цели
0.002 - SigmoidReversed Ложная тревога
------
0.729 - Normal Правильное обнаружение
0.117 - Normal Пропуск цели
0.154 - Normal Ложная тревога
------
0.857 - NormalFlipped Правильное обнаружение
0.143 - NormalFlipped Пропуск цели
0.001 - NormalFlipped Ложная тревога
------
0.751 - Plain Правильное обнаружение
0.12 - Plain Пропуск цели
0.129 - Plain Ложная тревога
------
0.8 - LinearIncrease Правильное обнаружение
0.13 - LinearIncrease Пропуск цели
0.07 - LinearIncrease Ложная тревога
------
0.852 - LinearDecrease Правильное обнаружение
0.142 - LinearDecrease Пропуск цели
0.006 - LinearDecrease Ложная тревога
------
-------------------------
0.847 - Sigmoid Правильное обнаружение
0.141 - Sigmoid Пропуск цели
0.012 - Sigmoid Ложная тревога
------
0.855 - SigmoidReversed Правил